# Acronyms
MIL - mother in law
FIL - father in law
W D - with dementia


# Untested Assumptions
1. Given sample of videos and channels, comments relate primarily to dementia, even if a dementia specific keyword is not used explicitly.
2. Comments are from real, knowledgeble informants, rather than bots or sarcastic posters
3. Commenters statements are truthful. For example, many delusions experienced by dementia patients relate to infidelity or theft. If a commenter asserts the dementia patient's claims are false, assume they are correct.


### Assumption 1 - Motivating Examples:
In the following comment, user responds to the content of the video, which outlined strategies for addressing delusions in dementia patients. The commenter never explicitly mentions dementia however.

```
I'm finding myself in more and more situations, where being honest with my loved one just makes matters worse.  There are times when he appears to have presence of mind and a pretty good understanding of reality.  It's during those episodes of clarity, when he tends to make statements or ask questions.  Where I find my replies of truth to be  like walking into a trap.  As if I just opened Pandora's box to his pent up hostility, anger and verbal abuse.  

Sometimes deception is not only to protect the feelings of the patient with dementia.  Sometimes  deception is necessary to  protect the caregivers.
```

Similarly, in the following comment, there is no discussion of dementia as a condition, or an underlying pathology. Based on context and knowledge of video content, it's clear the commenter is describing an individual with dementia.
```
Thank you for your videos.  The situation with my mom is now that she is older and has thinner skin she gets really cold.  She doesn’t believe this is why she gets colder.  She insists that we are the only people that has a cold house. Our temp is set around 72 or 73 degrees.  She says everyone else keeps there house temp at 80 degrees and she insists that we kept the house temp at 80 degrees year round for our whole lives.  Ex:  When my parents were in their 30’s and I was a young child she claims our house temp was always set at 80 degrees.  If you tell her it was not and that she gets colder now because of her age she gets really mad.  I should also mention this is not a once in a while conversation she has.  She talks about this multiple times every day.
```
### Assumption 2 - Motivating Examples:


### Assumption 3 - Motivating Examples:
There is no way to adjudicate truth in the following case. It's possible that the commentor (child) is unaware of their father's past history of infidelity. On the other hand, delusions relating to infidelity are common in dementia patients. For the purposes of classifying "possible delusion", we assume the knowledgable informant is correct.
```
My mom keeps thinking my father is out and about cheating on her, and mom sees him 24 hours a day.  He never goes out, but she still keeps thinking that.  Mom would confront dad, and of course he is not out cheating on her.  I honestly don't know how to handle this situation.
```


# Required models

### Coreferee (No longer using)
Installed "coreferee", instructions [here](https://github.com/richardpaulhudson/coreferee#the-basic-idea).

required admin elevantion to install

Required [installing Rust](https://www.rust-lang.org/tools/install) to get the fancy transformer-based model.

python -m spacy download en_core_web_lg

python -m spacy download en_core_web_trf

python -m coreferee install en


### neuralcoref
Hugging face state of the art...
```
pip install neuralcoref
pip install spacy[transformers]
pip install chardet
python -m spacy download en_core_web_lg
```

# The Gist
Vocabulary
Coreference
Anaphors - 


Why is it hard? Different syntactical structures in different languages. Some more explicit in identifying referents throughout a text.


In [2]:
import sqlite3
import os
conn = sqlite3.connect(os.path.join("..", "data", "comments2"))

# How to classify a dementia narrative by a knowledgeable informant?
A dementia narrative is a description of symptoms or behaviors of an individual with dementia by a knowledgeable informant.

This implies:
- Use of possessive when describing a "loved one", i.e. family member or friend.


Determining entities in a text and how they're described is "coreference resolution"

In [1]:
import spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp("Thank you for your videos. The situation with my mom is now that she is older and has thinner skin she gets really cold.  She doesn’t believe this is why she gets colder.  She insists that we are the only people that has a cold house. Our temp is set around 72 or 73 degrees.  She says everyone else keeps there house temp at 80 degrees and she insists that we kept the house temp at 80 degrees year round for our whole lives.  Ex:  When my parents were in their 30’s and I was a young child she claims our house temp was always set at 80 degrees.  If you tell her it was not and that she gets colder now because of her age she gets really mad.  I should also mention this is not a once in a while conversation she has.  She talks about this multiple times every day.")

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)


Thank thank VERB VBP ROOT Xxxxx True False
you -PRON- PRON PRP dobj xxx True True
for for ADP IN prep xxx True True
your -PRON- DET PRP$ poss xxxx True True
videos video NOUN NNS pobj xxxx True False
. . PUNCT . punct . False False
The the DET DT det Xxx True True
situation situation NOUN NN nsubj xxxx True False
with with ADP IN prep xxxx True True
my -PRON- DET PRP$ poss xx True True
mom mom NOUN NN pobj xxx True False
is be AUX VBZ ROOT xx True True
now now ADV RB advmod xxx True True
that that SCONJ IN mark xxxx True True
she -PRON- PRON PRP nsubj xxx True True
is be AUX VBZ ccomp xx True True
older old ADJ JJR acomp xxxx True False
and and CCONJ CC cc xxx True True
has have AUX VBZ conj xxx True True
thinner thin ADJ JJR amod xxxx True False
skin skin NOUN NN dobj xxxx True False
she -PRON- PRON PRP nsubj xxx True True
gets get VERB VBZ relcl xxxx True False
really really ADV RB advmod xxxx True True
cold cold ADJ JJ acomp xxxx True False
. . PUNCT . punct . False False
    SPACE 

Using coreferee library. Not impressed... The model is missing "I" and as a result, chain #3 incorrectly identifies "she", "her" etc. as referring to child (the commenter), rather than "mom".

In [2]:
# Load your usual SpaCy model (one of SpaCy English models)
import spacy
nlp = spacy.load('en_core_web_sm')

# Add neural coref to SpaCy's pipe
import neuralcoref
#neuralcoref.add_to_pipe(nlp)

neuralcoref.add_to_pipe(nlp, greedyness=0.5, max_dist=2500, blacklist=False) # first person pronouns are "blacklisted" by default

# You're done. You can now use NeuralCoref as you usually manipulate a SpaCy document annotations.
text = "My dad passed away last summer after suffering from the behavioral variant FTD, he was older when he was diagnosed but he probably had it longer but we probably missed a lot of signs over the years. When the symptoms really started getting bad that's when my dad's doctor ordered an MRI and we learned that the frontal part of his brain was shrinking and in atrophy.  For my dad, from diagnosis to death it took less than two years, a year and nine and a half months to be exact.    As another poster mentioned I wrote a journal of everything he went through and it was challenging for sure as my mother and I were his caregivers the entire time.  If there was one blessing it was that he never lost his memory of who my mom and I were so that was a good thing.  I could write a book here on what we went through with this disease but almost seven months since his passing, I wish I could take care of him for just one more day."
doc = nlp(text)

print(doc._.has_coref)
print(doc._.coref_clusters)
print(doc._.coref_resolved)

True
[My: [My, my, my, I, my, I, my, I, I, I, I], My dad: [My dad, he, he, he, my dad, my dad], we: [we, we, we], my dad's doctor: [my dad's doctor, his, he, his, he, his, his, him]]
My dad passed away last summer after suffering from the behavioral variant FTD, My dad was older when My dad was diagnosed but My dad probably had it longer but we probably missed a lot of signs over the years. When the symptoms really started getting bad that's when My dad's doctor ordered an MRI and we learned that the frontal part of my dad's doctor brain was shrinking and in atrophy.  For My dad, from diagnosis to death it took less than two years, a year and nine and a half months to be exact.    As another poster mentioned My wrote a journal of everything my dad's doctor went through and it was challenging for sure as My mother and My were my dad's doctor caregivers the entire time.  If there was one blessing it was that my dad's doctor never lost my dad's doctor memory of who My mom and My were so t

In [5]:
# Performs poorly on excerpts where possessive pronoun "my" does not precede the patient identifier (e.g. "my mom")
text = "Dr. Allison, mom is behaving the opposite way. She lives in an assisted living facility and she is pushing the call button every few minutes to have them hand her the remote when it is sitting right next to her, wanting them to wipe her when she goes to the bathroom and many other things like that. The caregivers are so frustrated and the nurse is trying to get her to do these things for herself while she still can. I am frustrated and can’t be around her because I am just so exhausted and I feel like her slave. What is your advice?"
doc = nlp(text)

print(doc._.has_coref)
print(doc._.coref_clusters)
print(doc._.coref_resolved)

True
[Dr. Allison: [Dr. Allison, She, she, her, her, her, she, her, herself, she, I, her, I, I, her, your], them: [them, them], the remote: [the remote, it]]
Dr. Allison, mom is behaving the opposite way. Dr. Allison lives in an assisted living facility and Dr. Allison is pushing the call button every few minutes to have them hand Dr. Allison the remote when the remote is sitting right next to Dr. Allison, wanting them to wipe Dr. Allison when Dr. Allison goes to the bathroom and many other things like that. The caregivers are so frustrated and the nurse is trying to get Dr. Allison to do these things for Dr. Allison while Dr. Allison still can. Dr. Allison am frustrated and can’t be around Dr. Allison because Dr. Allison am just so exhausted and Dr. Allison feel like Dr. Allison slave. What is Dr. Allison advice?


In [7]:
# May need to resolve acronymns... LO = Loved One. MIL = Mother in Law
# Understanding why "I" and "my" are blacklisted.
text = """
Yes but it all depends on the state of dementia of your LO. Anything I try to say, request her to do or discuss with my mother is met with hostility. She is deeply paranoid and suspicious. She makes unfounded and quite horrid accusations to and about me. She argues with me even when I am being nice to her. She argues about her arguing!!! Bascially I am her punch bag and it is soul destroying.

If you haven't already get a PoA and do what you have to do without them making it harder for you. If day to day tasks become impossible to complete then get home help for assistance or check your LO into a care facility. There comes a point when the stress and aggravation is just not worth it. They have dementia what do they know? Many times I feel like my mother is frying my brain and it literally hurts my head and I just want out. Can't do it any more, but I am trapped and she knows it because as well as having dementia she is a manipulative selfish narcissist which is a terrible combination. Get help is all I can say. Trying to deal with contentious issues on a one to one basis with a LO who is resistant non compliant is, 99% of the time, going to fail."""
doc = nlp(text)

print(doc._.has_coref)
print(doc._.coref_clusters)
print(doc._.coref_resolved)

True
[the state of dementia of your LO: [the state of dementia of your LO, her, She, She, She, her, She, her, her], your LO: [your LO, your LO, LO, LO], I: [I, my, me, me, I, I, I, my, my, my, I, I, I], my mother: [my mother, my mother, she, she], her punch bag: [her punch bag, it], you: [you, you], They: [They, they], it: [it, it, it]]

Yes but it all depends on the state of dementia of your LO. Anything I try to say, request the state of dementia of your LO to do or discuss with I mother is met with hostility. the state of dementia of your LO is deeply paranoid and suspicious. the state of dementia of your LO makes unfounded and quite horrid accusations to and about I. the state of dementia of your LO argues with I even when I am being nice to the state of dementia of your LO. the state of dementia of your LO argues about the state of dementia of your LO arguing!!! Bascially I am the state of dementia of your LO punch bag and her punch bag is soul destroying.

If you haven't already 

In [8]:
# Father resolves to "God". Yikes!
text = "I lost my Mother to dementia the 15th of December 2021. I always watched your videos, because her doctor's could not help me. The only answers I got were no one really knows how it works. Through your videos I was more able to help my Mother and my self. I really appreciate you being there for me. I kept a journal of my journey with my Mother. I found playing games with her a great help when she had a bad day. When she didn't want to take a bath or get up to go to the bathroom, I would play a game, we would go to the bathroom and quack and walk like a duck. I would make up silly games to get her to eat. The hardest part for me was when hospice came in and she finally could not walk and she ended up in a hospital bed. I still played games with her to get what little bit of food I could get in her. I tried to make every day count. I still miss her but when she left me, I was glad for her. She will never hurt or suffer again. I know she was happy because she wanted to be with my Father. Tomorrow my Father will be gone for 4yrs. I took care of him when he had bladder cancer.  I lived with my parents for 10 yrs and I am so thankful God gave me that time.  I hope each and every one who is taking care of a love one has the precious memories I have and enjoys their special time with them. Thank you so much for your help and God bless you!"
doc = nlp(text)
print(doc._.has_coref)
print(doc._.coref_clusters)
print(doc._.coref_resolved)

True
[I: [I, my, I, me, I, I, my, my, I, me, I, my, my, I, I, I, me, I, I, I, I, me, I, I, my, my, I, I, my, I, me, I, I, you, your, you], my Mother: [my Mother, her, my Mother, her, she, she, her, she, she, her, her, her, she, her, She, she, she], your videos: [your videos, your videos], no one: [no one, it], your: [your, you], my Father: [my Father, my Father, him, he, God, God], their: [their, them]]
I lost my Mother to dementia the 15th of December 2021. I always watched your videos, because my Mother doctor's could not help I. The only answers I got were no one really knows how no one works. Through your videos I was more able to help I Mother and I self. I really appreciate your being there for I. I kept a journal of I journey with I Mother. I found playing games with my Mother a great help when my Mother had a bad day. When my Mother didn't want to take a bath or get up to go to the bathroom, I would play a game, we would go to the bathroom and quack and walk like a duck. I woul

In [9]:
text = """
We embraced my husband's dementia because.... it is what it is.   When he first got the diagnosis we were mostly relieved, better than the dramas and psychosis that plagued him for a few years.   That had been a miserable time and I was his target.   Knowing what it was,  made our lives better.

We ended up with a good medical team, the right meds after a few months of trials, government pension,  did all the legal paperwork while he could still function and we told people what he had with no shame or hesitation.

7 years down with him at a moderate to severe stage and him sitting most days with his own thoughts, I think maybe his life is not so bad, no decisions, no bills, no driving, no responsibility,  not answerable for anything...... perpetual holiday of the mind.  It's then I think he's the lucky one.   I just get the work.
"""
doc = nlp(text)
print(doc._.has_coref)
print(doc._.coref_clusters)
print(doc._.coref_resolved)

True
[We: [We, we, our, We, we], my: [my, I, I, I, I], my husband: [my husband, he, him, his, he, he, him, him, his, his, he]]

We embraced my husband's dementia because.... it is what it is.   When my husband first got the diagnosis We were mostly relieved, better than the dramas and psychosis that plagued my husband for a few years.   That had been a miserable time and my was my husband target.   Knowing what it was,  made We lives better.

We ended up with a good medical team, the right meds after a few months of trials, government pension,  did all the legal paperwork while my husband could still function and We told people what my husband had with no shame or hesitation.

7 years down with my husband at a moderate to severe stage and my husband sitting most days with my husband own thoughts, my think maybe my husband life is not so bad, no decisions, no bills, no driving, no responsibility,  not answerable for anything...... perpetual holiday of the mind.  It's then my think my hu

In [10]:
text = """
Thank you for your tireless advocation and tips, Dr. Natali! Not only do I work in memory care, but I just lost my father to vascular dementia a week ago. I have thoroughly enjoyed your videos and sent them to my family so they could have a better relationship with Dad with the time he had left over the last 2 years. I am determined to do all I can to keep this education going and pay it forward. It is shocking how many people come to me sharing a loved one's diagnosis and asking for advice. I always direct them here, among other resources. It seems to be a growing epidemic which breaks my heart. :(
"""
doc = nlp(text)
print(doc._.has_coref)
print(doc._.coref_clusters)
print(doc._.coref_resolved)



True
[you: [you, your, I, I, my, I, my, I, I, me, I, my], my father: [my father, he], your videos: [your videos, them, they], this education: [this education, it], how many people: [how many people, them]]

Thank you for you tireless advocation and tips, Dr. Natali! Not only do you work in memory care, but you just lost you father to vascular dementia a week ago. you have thoroughly enjoyed your videos and sent your videos to you family so your videos could have a better relationship with Dad with the time my father had left over the last 2 years. you am determined to do all you can to keep this education going and pay this education forward. It is shocking how many people come to you sharing a loved one's diagnosis and asking for advice. you always direct how many people here, among other resources. It seems to be a growing epidemic which breaks you heart. :(



In [11]:
text = "I had a stroke many years ago and was diagnosed with dementia. I am living my life with my life and pets, cats and dogs. I  believe I am almost normal now but I have problems with my wife. I balieve you need to use your brain, memory and intelligence to continue living your life otherwise you will lose it. Just behave normally and you can regain all your memory."
doc = nlp(text)
print(doc._.has_coref)
print(doc._.coref_clusters)
print(doc._.coref_resolved)


True
[I: [I, I, my, my, I, I, I, my, I], your life: [your life, it]]
I had a stroke many years ago and was diagnosed with dementia. I am living I life with I life and pets, cats and dogs. I  believe I am almost normal now but I have problems with I wife. I balieve you need to use your brain, memory and intelligence to continue living your life otherwise you will lose your life. Just behave normally and you can regain all your memory.


In [12]:
text = """Thank you for all your fine commentary.   H0w do we cope with a 2-fer as  Teepa  has previously brought up. 
We have 2  of 90 year old parents,  mom with serious dementia and a father determined to be her caregiver,
yet his dementia gets worse by the day and is determined to keep everything the way it was."""
doc = nlp(text)
print(doc._.has_coref)
print(doc._.coref_clusters)
print(doc._.coref_resolved)

True
[you: [you, your], we: [we, We], a father: [a father, her, his]]
Thank you for all you fine commentary.   H0w do we cope with a 2-fer as  Teepa  has previously brought up. 
we have 2  of 90 year old parents,  mom with serious dementia and a father determined to be a father caregiver,
yet a father dementia gets worse by the day and is determined to keep everything the way it was.


# Suggested Algorithm
1. Remove sentences that only express gratitude
2. Find instances where length of coreference chain of "my xxx" is > than instances of "I"

### Gratitude
God bless you
Thank*
Ty